In [ ]:
!pip install spacy
!pip install transformers
!pip install spacy-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
import json
import spacy
from spacy.tokens import DocBin, Span

def load_data(file_path):
    """Load JSON data from file."""
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def convert_to_spacy(data, nlp):
    """Convert JSON NER dataset into spaCy's binary format while checking index errors."""
    doc_bin = DocBin()

    for entry in data:
        text = entry["text"]
        doc = nlp.make_doc(text)

        entities = []
        for ent in entry["entity"]:
            start_char = ent["start"]
            end_char = ent["end"]
            label = ent["label"]


            if start_char >= len(text) or end_char > len(text) or start_char >= end_char:
                print(f"Skipping invalid entity: '{label}' in text: '{text}' (start={start_char}, end={end_char})")
                continue

            entity_text = text[start_char:end_char]
            char_span = doc.char_span(start_char, end_char, label=label)

            if char_span is not None:
                entities.append(char_span)
            else:
                print(f"Skipping entity '{entity_text}' in text '{text}' due to tokenization issues.")

        doc.ents = entities
        doc_bin.add(doc)  #

    return doc_bin

nlp = spacy.load("en_core_web_sm")

for split in ["train", "test", "validation"]:
    data = load_data(f"/content/drive/MyDrive/End to End NER project/NER_{split}_data.json")
    doc_bin = convert_to_spacy(data, nlp)
    doc_bin.to_disk(f"{split}.spacy")
    doc_bin.to_disk(f"/content/drive/MyDrive/End to End NER project/NER_{split}_spacy_data.spacy")


!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./validation.spacy


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0         77.21     78.91    1.80    1.11    4.82    0.02
  0     200       2330.46   3402.05   50.18   57.50   44.51    0.50
  0     400       1061.75   2201.30   65.04   67.18   63.03    0.65
  0     600       3355.01   2239.02   68.76   70.86   66.78    0.69
  0     800       1063.72   2200.95   72.34   74.62   70.18    0.72
  0    1000       1617.47   2424.69   74.54   74.96   74.13    0.75
  1    1200       2646.95   2401.54   76.70   77.56   75.87    0.77
  1    1400       1878.79   1883.33   77.85   77.97   77.74    0

In [ ]:
import spacy
import json
from spacy.training import Example
from spacy.scorer import Scorer

nlp = spacy.load("./output/model-best")

def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

test_data = load_data("/content/drive/MyDrive/End to End NER project/NER_test_data.json")

examples = []
for entry in test_data:
    text = entry["text"]
    entities = [(ent["start"], ent["end"], ent["label"]) for ent in entry["entity"]]

    gold_doc = nlp.make_doc(text)
    gold_ents = []

    for start, end, label in entities:
        span = gold_doc.char_span(start, end, label=label)
        if span is not None:
            gold_ents.append(span)

    gold_doc.ents = gold_ents
    pred_doc = nlp(text)

    example = Example(pred_doc, gold_doc)
    examples.append(example)

scorer = Scorer()
scores = scorer.score(examples)

print("Precision:", scores["ents_p"])
print("Recall:", scores["ents_r"])
print("F1-score:", scores["ents_f"])
print("Entity Type Breakdown:", scores["ents_per_type"])


In [ ]:
nlp.to_disk("/content/drive/MyDrive/End to End NER project/spacy_model1")